In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 25.12 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [ ]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader) #moving the pointer to 2nd row
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [ ]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [ ]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [ ]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/training', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/testing', emotion, image)
    copyfile(source, dest)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = tf.keras.models.Sequential([
      Conv2D(128, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(256, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(256, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(512, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dropout(0.2),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 98, 98, 128)       3584      
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 47, 47, 256)       295168    
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 23, 23, 256)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 21, 21, 256)       590080    
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 10, 10, 256)       0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 8, 8, 512)        

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100,100),
                                                    class_mode = 'categorical',
                                                    batch_size = 64
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100,100),
                                                    class_mode = 'categorical',
                                                    batch_size = 64
                                                  )

Found 10941 images belonging to 8 classes.
Found 4521 images belonging to 8 classes.


In [ ]:
es = EarlyStopping(monitor='val_acc', patience = 5, min_delta=0.01)

In [ ]:
model.fit_generator(train_generator,
                    epochs = 30,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
171/171 [==============================] - 28s 147ms/step - loss: 39.8715 - acc: 0.4723 - val_loss: 1.0533 - val_acc: 0.5014
Epoch 2/30
171/171 [==============================] - 23s 133ms/step - loss: 1.0416 - acc: 0.4836 - val_loss: 1.0263 - val_acc: 0.5021
Epoch 3/30
171/171 [==============================] - 22s 130ms/step - loss: 1.0219 - acc: 0.5050 - val_loss: 0.9927 - val_acc: 0.5800
Epoch 4/30
171/171 [==============================] - 22s 127ms/step - loss: 0.9234 - acc: 0.6204 - val_loss: 0.8816 - val_acc: 0.6642
Epoch 5/30
171/171 [==============================] - 22s 126ms/step - loss: 0.7780 - acc: 0.7215 - val_loss: 0.8537 - val_acc: 0.6881
Epoch 6/30
171/171 [==============================] - 21s 125ms/step - loss: 0.7254 - acc: 0.7434 - val_loss: 0.8674 - val_acc: 0.6806
Epoch 7/30
171/171 [==============================] - 22s 127ms/step - loss: 0.7129 - acc: 0.7466 - val_loss: 0.9456 - val_acc: 0.6786
Epoch 8/30
171/171 [==============================] - 